In [2]:
import glob
from Bio import SeqIO
import pandas as pd
import numpy as np
import os

out_seqs = []
records = []

for file in glob.glob('/disk2/projects/TempertonLab/SAR202/data/fasta/*.fasta'):
    seq_name = os.path.basename(file).replace('.fasta', '')
    with open(file, 'r') as handle:
        seq_lengths = []
        for record in SeqIO.parse(handle, 'fasta'):
            record.id = '%s_%s' % (seq_name, record.id)
            record.description=''
            seq_lengths.append(len(record.seq))
            out_seqs.append(record)
        records.append((seq_name, len(seq_lengths), np.sum(seq_lengths)/1e6, np.min(seq_lengths), np.max(seq_lengths)))

SeqIO.write(out_seqs, '/disk2/projects/TempertonLab/SAR202/scratch/combined.seqs.fa', 'fasta')
df = pd.DataFrame(records, columns=['seq_name', 'number_of_contigs', 'total_length_Mbp', 'shortest_contig', 'longest_contig_length'])
df = df.sort_values('total_length_Mbp', ascending=False)
df

,seq_name,number_of_contigs,total_length_Mbp,shortest_contig,longest_contig_length
4,MPNC01,67,4.958136,1702,757257
17,MPND01,213,4.606320,1084,169007
62,MPMM01,58,4.393250,1778,1117843
27,OSU_TB21,251,3.950169,2507,106794
90,OSU_TB15,299,3.936920,2511,67435
36,MPMJ01,101,3.223809,1149,202044
73,OSU_TB65,82,3.121826,2553,312586
115,OSU_TB22,69,3.094065,2952,284945
61,MPNN01,21,3.017767,3194,507178
69,UM_GB_B1,73,2.998127,5396,329954


Now we run VirSorter

In [ ]:
!cd /disk2/projects/TempertonLab/SAR202/scratch
!wrapper_phage_contigs_sorter_iPlant.pl \
-f combined.seqs.fa \
--db 2 \
--wdir virsorter \
--ncpu 16 \
--data-dir /disk2/dbs/virsorter-data

In [15]:
import pandas as pd
import numpy as np

out_phage = open('/disk2/projects/TempertonLab/SAR202/scratch/phage.csv', 'w')
out_prophage = open('/disk2/projects/TempertonLab/SAR202/scratch/prophage.csv', 'w')
phage=True

#split the virsorter output into two sections so we can re-label 1-3 with 4-6 for prophage
with open('/disk2/projects/TempertonLab/SAR202/scratch/virsorter/VIRSorter_global-phage-signal.csv', 'r') as handle:
    for line in handle.readlines():
        if line.startswith('## 4'):
            phage=False
        if phage:
            out_phage.write(line)
        else:
            out_prophage.write(line)
out_phage.close()
out_prophage.close()
            

phage_df = pd.read_csv('/disk2/projects/TempertonLab/SAR202/scratch/phage.csv',
                          names=['contig',
                                'num_genes',
                                 'fragment',
                                'virsorter_category'],
                          usecols=[0, 1,2,4],
                          comment='#')

prophage_df = pd.read_csv('/disk2/projects/TempertonLab/SAR202/scratch/prophage.csv',
                          names=['contig',
                                'num_genes',
                                 'fragment',
                                'virsorter_category'],
                          usecols=[0, 1,2,4],
                          comment='#')

prophage_df['virsorter_category'] = prophage_df['virsorter_category'] + 3

def is_circular(row):
    return 'circular' in row['contig']

virsorter_df = phage_df.append(prophage_df).reset_index().drop(columns=['index'])
virsorter_df['is_circular'] = virsorter_df.apply(is_circular, axis=1)
virsorter_df['contig'] = virsorter_df['contig'].replace([r'VIRSorter_(.*_cov_\d+)_(\d+).*', r'VIRSorter_',
                                                        r'(BSAE-\d{4}-\d)_([A-Z0-9]+)', r'-circular'], 
                                                        [r'\1.\2', '', r'\1.\2', ''], regex=True)
virsorter_df['fragment'] = virsorter_df['fragment'].replace([r'VIRSorter_(.*_cov_\d+)_(\d+).*', 
                                                             r'VIRSorter_', 
                                                             r'(BSAE-\d{4}-\d)_([A-Z0-9]+)', 
                                                            r'-circular'], 
                                                        [r'\1.\2', '', r'\1.\2', ''], regex=True)

!rm /disk2/projects/TempertonLab/SAR202/scratch/phage.csv /disk2/projects/TempertonLab/SAR202/scratch/prophage.csv
virsorter_df[~virsorter_df['is_circular']]
virsorter_df

,contig,num_genes,fragment,virsorter_category,is_circular
0,USC_TB68_TB68_001,10,USC_TB68_TB68_001,1,False
1,USC_TB72_TB72_055,40,USC_TB72_TB72_055,1,False
2,LSU_GOM1_GOM1_103,12,LSU_GOM1_GOM1_103,2,False
3,LSU_GOM1_GOM1_176,13,LSU_GOM1_GOM1_176,2,False
4,MPMJ01_MPMJ_030,51,MPMJ01_MPMJ_030,2,False
5,MPMJ01_MPMJ_042,22,MPMJ01_MPMJ_042,2,False
6,MPMJ01_MPMJ_057,18,MPMJ01_MPMJ_057,2,False
7,MPMM01_MPMM_027,50,MPMM01_MPMM_027,2,True
8,MPND01_MPND_161,5,MPND01_MPND_161,2,False
9,MPNN01_MPNN_009,114,MPNN01_MPNN_009,2,True


Now we run VirFinder

In [ ]:
!cd /disk2/projects/TempertonLab/SAR202/scratch
!mkdir virfinder
!mkdir tmp
!python /home/bt273/tools/scripts/split_reads.py \
--in_file /disk2/projects/TempertonLab/SAR202/scratch/combined.seqs.fa \
--out_dir tmp \
--reads_per_file 1000

!for i in ./tmp/group*.fa; \
do OUTNAME=$(basename $i .fa); \
OUTNAME+=.csv; \
echo "$i,virfinder/$OUTNAME" >> to_parse; \
done

!parallel --colsep ',' /home/bt273/tools/scripts/runVirFinder.R {1} {2} :::: to_parse
!/home/bt273/tools/scripts/combineVirFinder.R virfinder combined.seqs.virfinder.csv
!rm -rf virfinder tmp

In [16]:
import pandas as pd
import numpy as np
virfinder_df = pd.read_csv('/disk2/projects/TempertonLab/SAR202/scratch/combined.seqs.virfinder.csv', 
                           names=['contig', 
                                                                              'length',
                                                                              'virfinder_score',
                                                                             'virfinder_pvalue'],
                          skiprows=1)

In [17]:
viral_df = virfinder_df.merge(virsorter_df, how='left', on='contig')
def is_viral(row):
    return ((row['virsorter_category']==1 or row['virsorter_category']==2) or
        (row['virfinder_score'] > 0.9 and row['virfinder_pvalue'] <0.05) or
        (row['virfinder_score'] >0.7 and 
             row['virfinder_pvalue'] <0.05 and
            ~np.isnan(row['virsorter_category'])))
        
viral_df['is_viral'] = viral_df.apply(is_viral, axis=1)
viral_df.to_csv('/disk2/projects/TempertonLab/SAR202/data/combined.seqs.viral.check.csv', index=False)

only_viral=viral_df[viral_df['is_viral']].sort_values('length', ascending=False)
print("There are %i contigs that have been identified as viral" % len(only_viral.contig.values))

There are 402 contigs that have been identified as viral


In [20]:
only_viral.to_csv('/disk2/projects/TempertonLab/SAR202/data/viral.contigs.csv')

In [29]:
only_viral.head()
print("""
In total there are %i contigs that are high-confidence viral (Virsorter 1245 >10kb).
There are %i contigs > 10kb.""" % (
    only_viral[(only_viral.virsorter_category.isin([1,2,4,5])) &
                 (only_viral.length > 10000)].shape[0], 
      only_viral[only_viral.length > 10000].shape[0]))


In total there are 24 contigs that are high-confidence viral (Virsorter 1245 >10kb).
There are 52 contigs > 10kb.


In [27]:
402 - only_viral[only_viral.length > 5000].shape[0]

227

In [31]:
only_viral[(only_viral.virsorter_category.isin([1,2,4,5])) &
                 (only_viral.length > 10000)]

,contig,length,virfinder_score,virfinder_pvalue,num_genes,fragment,virsorter_category,is_circular,is_viral
4007,OSU_TB57_TB57_007,152655,0.952741,0.006791,171.0,OSU_TB57_TB57_007,2.0,False,True
4003,OSU_TB57_TB57_009,93671,0.972695,0.004608,68.0,OSU_TB57_TB57_009,2.0,False,True
12694,MPNN01_MPNN_009,89568,0.898420,0.013461,114.0,MPNN01_MPNN_009,2.0,True,True
10686,USC_TB94_TB94_020,52670,0.912485,0.012248,67.0,USC_TB94_TB94_020,2.0,False,True
1003,OSU_TB07_TB07_119,49618,0.969444,0.005336,92.0,OSU_TB07_TB07_119,2.0,False,True
12713,USC_TB67_TB67_058,46377,0.842318,0.022678,68.0,USC_TB67_TB67_058,2.0,False,True
9915,MPMJ01_MPMJ_030,39372,0.558021,0.075431,51.0,MPMJ01_MPMJ_030,2.0,False,True
2004,OSU_ZL03_ZL03_105,38812,0.987282,0.002789,56.0,OSU_ZL03_ZL03_105,2.0,False,True
12723,MPNN01_MPNN_017,36743,0.815375,0.026195,31.0,MPNN01_MPNN_017,2.0,False,True
12959,MPMM01_MPMM_027,34861,0.422834,0.111691,50.0,MPMM01_MPMM_027,2.0,True,True


I think that there is evidence here that VirFinder is struggling with 'new', even in short-read data, so for now, I want to focus on the analysis of the 24 viral contigs identified by VirSorter, and maybe we can come back to those others later. It's very cool to see that OSU_ZL03_ZL03_105 is a phage from a single cell from Zach's work.

Let's pull out the data we need:

In [37]:
from Bio import SeqIO

hq_sar202_phage = only_viral[(only_viral.virsorter_category.isin([1,2,4,5])) &
                 (only_viral.length > 10000)]

seqs = SeqIO.index('/disk2/projects/TempertonLab/SAR202/scratch/combined.seqs.fa', 'fasta')
out_seqs = [seqs[x] for x in hq_sar202_phage.contig.values]
#len(out_seqs)
SeqIO.write(out_seqs, '/disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.fa', 'fasta')

24

In [54]:
out_gbk = []
! sed -i "s/VIRSorter_//g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-1.gb
! sed -i "s/-circular//g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-1.gb

! sed -i "s/VIRSorter_//g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-2.gb
! sed -i "s/-circular//g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-2.gb
! sed -i "s/07\/24\/18/24-Jul-2018/g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-1.gb
! sed -i "s/07\/24\/18/24-Jul-2018/g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-2.gb
! sed -i "s/dna/DNA/g" /disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-2.gb

for record in SeqIO.parse('/disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-1.gb', 'genbank'):
    if record.id in hq_sar202_phage.contig.values:
        out_gbk.append(record)

for record in SeqIO.parse('/disk2/projects/TempertonLab/SAR202/scratch/virsorter/Predicted_viral_sequences/VIRSorter_cat-2.gb', 'genbank'):
    if record.id in hq_sar202_phage.contig.values:
        out_gbk.append(record)

SeqIO.write(out_gbk, '/disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.gbk', 'genbank')

/home/bt273/anaconda3/lib/python3.6/site-packages/Bio/GenBank/Scanner.py:1368: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       USC_TB72_TB72_055        29269 bp    dna     linear   ENV 24-07-2018\n'
Found locus 'USC_TB72_TB72_055' size '29269' residue_type 'dna'
Some fields may be wrong.
  BiopythonParserWarning)
/home/bt273/anaconda3/lib/python3.6/site-packages/Bio/GenBank/Scanner.py:1368: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       USC_TB68_TB68_001        12111 bp    dna     linear   ENV 24-07-2018\n'
Found locus 'USC_TB68_TB68_001' size '12111' residue_type 'dna'
Some fields may be wrong.
  BiopythonParserWarning)
/home/bt273/anaconda3/lib/python3.6/site-packages/Bio/GenBank/Scanner.py:1368: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       OSU_TB57_TB57_007       152655 bp    DNA     linear   ENV 24-Jul-2018\n'
Found locus 'OSU_TB57_TB57_007' size '152655' residue_type 'DNA'
Some fields 

24

Now I need to the viral predicted protein sequences and blasting them against NR to work out both their function and their taxonomy


In [63]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
prots = []
for record in SeqIO.parse('/disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.gbk','genbank'):
    if record.features:
        for feature in record.features:
            if feature.type == "CDS":
                prots.append(SeqRecord(
                    feature.location.extract(record).seq, 
                    id=feature.qualifiers['locus_tag'][0], description=''))
SeqIO.write(prots, '/disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.prots.faa', 'fasta')

1055

Now we can run these against NR

In [ ]:
!diamond blastp --threads 16 \
--db /disk2/dbs/ncbi/nr.dmnd \
--out /disk2/projects/TempertonLab/SAR202/scratch/sar202.viral.contigs.10k.cat12.prots.vs.nr.out.gz \
--outfmt 6 qseqid sseqid stitle staxids pident length qstart qend sstart send evalue \
--query /disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.prots.faa \
-k 500 \
--more-sensitive \
--compress 1 \
--subject-cover 20 \
--evalue 1e-5

# Annotating with Prokka

We want to run prokka on each contig individually, then recombine them so that we can keep the contig name and the locus name sensible.

We're going to annotate them against dsDNA phage genomes that are known to infect bacteria:
```bash
Viruses[Organism] AND srcdb_refseq[PROP] NOT wgs[PROP] NOT cellular organisms[ORGN] NOT AC_000001:AC_999999[PACC] AND ("vhost bacteria"[Filter]) 
```

```bash
prokka --outdir /disk2/projects/TempertonLab/SAR202/data/prokka \
--force \
--kingdom Viruses \
--addgenes \
--mincontiglen 200 \
--centre XXX \
--metagenome \
--cdsrnaolap \
--cpus 16 \
--rfam \
--proteins /disk2/dbs/ncbi/refseq/refseq.viral.v85.gpff \
/disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.fa
```

In [72]:
from Bio import SeqIO
import subprocess


for record in SeqIO.parse('/disk2/projects/TempertonLab/SAR202/data/sar202.viral.contigs.10k.cat12.fa', 'fasta'):
    SeqIO.write([record], 'tmp.fa', 'fasta')
    cmd = """
    prokka --outdir /disk2/projects/TempertonLab/SAR202/scratch/%s --force \
    --compliant \
    --prefix %s \
    --centre Z \
    --kingdom Virus \
    --metagenome \
    --cdsrnaolap \
    --cpus 16 \
    --rfam \
    --genus "Putative_SAR202_phage" \
    --locustag %s_gene_ \
    --proteins /disk2/dbs/ncbi/refseq/dsDNA.phage.gb \
    --hmms /disk2/dbs/vFAM/vFam-B_2014.hmm \
    tmp.fa
    """ % (record.id, record.id, record.id)
    subprocess.call(cmd.split())
    